In [1]:
import datetime

import csv

import pandas as pd

from sqlalchemy import create_engine

import yfinance as yf 

from bs4 import BeautifulSoup

import requests

from pyspark.sql import SparkSession

import pyspark

from pyspark.context import SparkContext

import pyspark.sql.functions as fc

import pymysql

from pyspark.sql import Row

from pyspark.sql.types import *

import json

import numpy as np

In [2]:
sc = SparkContext.getOrCreate()

spark = SparkSession(sc)

In [3]:
stockspark = spark.read.json('stockdata.json')

companyspark = spark.read.json('companyinfo.json')

In [6]:
companyspark.show(10)

+--------------------+--------------------+------+
|                Name|              Sector|Symbol|
+--------------------+--------------------+------+
|Agilent Technolog...|         Health Care|     A|
|American Airlines...|         Industrials|   AAL|
|  Advance Auto Parts|Consumer Discreti...|   AAP|
|          Apple Inc.|Information Techn...|  AAPL|
|         AbbVie Inc.|         Health Care|  ABBV|
|AmerisourceBergen...|         Health Care|   ABC|
|         ABIOMED Inc|         Health Care|  ABMD|
| Abbott Laboratories|         Health Care|   ABT|
|       Accenture plc|Information Techn...|   ACN|
|          Adobe Inc.|Information Techn...|  ADBE|
+--------------------+--------------------+------+
only showing top 10 rows



In [7]:
stockspark.show(10)

+-----+----------+---------+-----+-----+----+-----+---------+
|close|      date|dividends| high|  low|name| open|   volume|
+-----+----------+---------+-----+-----+----+-----+---------+
| 85.4|01/02/2020|      0.0| 85.8|84.65|   A|85.35|1410500.0|
|84.03|01/03/2020|      0.0|84.78|83.96|   A|84.13|1118300.0|
|84.28|01/06/2020|      0.0|84.28|83.06|   A|83.46|1993200.0|
|84.53|01/07/2020|      0.0|84.71| 83.4|   A|83.42|1684700.0|
|85.37|01/08/2020|      0.0|85.91|84.65|   A|85.41|1847600.0|
|86.71|01/09/2020|      0.0|87.14|85.62|   A| 85.9|1912700.0|
|87.03|01/10/2020|      0.0|87.67|86.76|   A|87.16|1417000.0|
| 86.9|01/13/2020|      0.0|87.75|86.18|   A|87.25|1630200.0|
|87.43|01/14/2020|      0.0|87.64|86.14|   A|86.71|1675200.0|
|88.05|01/15/2020|      0.0|88.54|86.99|   A|87.07|1630400.0|
+-----+----------+---------+-----+-----+----+-----+---------+
only showing top 10 rows



In [8]:
sectorjoin=stockspark.join(companyspark,stockspark.name==companyspark.Symbol).select(stockspark.date,companyspark.Symbol,stockspark.open,stockspark.close,stockspark.high,stockspark.low,stockspark.volume,stockspark.dividends,companyspark.Sector)

In [9]:
sectorjoin.show()

+----------+------+-----+-----+-----+-----+---------+---------+-----------+
|      date|Symbol| open|close| high|  low|   volume|dividends|     Sector|
+----------+------+-----+-----+-----+-----+---------+---------+-----------+
|01/02/2020|     A|85.35| 85.4| 85.8|84.65|1410500.0|      0.0|Health Care|
|01/03/2020|     A|84.13|84.03|84.78|83.96|1118300.0|      0.0|Health Care|
|01/06/2020|     A|83.46|84.28|84.28|83.06|1993200.0|      0.0|Health Care|
|01/07/2020|     A|83.42|84.53|84.71| 83.4|1684700.0|      0.0|Health Care|
|01/08/2020|     A|85.41|85.37|85.91|84.65|1847600.0|      0.0|Health Care|
|01/09/2020|     A| 85.9|86.71|87.14|85.62|1912700.0|      0.0|Health Care|
|01/10/2020|     A|87.16|87.03|87.67|86.76|1417000.0|      0.0|Health Care|
|01/13/2020|     A|87.25| 86.9|87.75|86.18|1630200.0|      0.0|Health Care|
|01/14/2020|     A|86.71|87.43|87.64|86.14|1675200.0|      0.0|Health Care|
|01/15/2020|     A|87.07|88.05|88.54|86.99|1630400.0|      0.0|Health Care|
|01/16/2020|